In [52]:
import numpy as np
import functools as ft
import torch

In [53]:
torch.set_default_device("cuda" if torch.cuda.is_available() else "cpu")

In [57]:
def hdv(d):
    return np.random.choice([-1, 1], d)


def hdvt(d):
    return torch.sign(torch.randint(-100000, 100000, (d,), dtype=torch.float))


def bind(xs):
    return ft.reduce(lambda x, y: x * y, xs)


def bindt(xs):
    return torch.prod(torch.cat(xs), axis=0)


def bundle(xs):
    return ft.reduce(lambda x, y: x + y, xs)


def bundlet(xs):
    return torch.sum(torch.cat(xs), axis=0)


def cosine_similarity(A, B):
    dot_product = np.dot(A, B)
    norm_A = np.linalg.norm(A)
    norm_B = np.linalg.norm(B)

    if norm_A == 0 or norm_B == 0:
        return 0

    return dot_product / (norm_A * norm_B)


def cosine_similarityt(A, B):
    dot_product = torch.dot(A, B)
    norm_A = torch.norm(A)
    norm_B = torch.norm(B)

    # if norm_A == 0 or norm_B == 0:
    #     return 0

    return torch.div(dot_product, norm_A * norm_B)

In [55]:
class ItemMemory:
    def __init__(self, vectors=[]):
        self.vectors = vectors

    def addVector(self, label, V):
        self.vectors.append((label, V))

    def cleanup(self, V):
        return max(self.vectors, key=lambda x: cosine_similarity(V, x[1]))

In [58]:
import timeit

print("hdv ", timeit.timeit("hdv(10000)", number=10000, globals=globals()))
print("hdvt", timeit.timeit("hdvt(10000)", number=10000, globals=globals()))

print(
    "bin ",
    timeit.timeit("bind([hdv(10000), hdv(10000)])", number=10000, globals=globals()),
)
print(
    "bint",
    timeit.timeit("bindt([hdvt(10000), hdvt(10000)])", number=10000, globals=globals()),
)

print(
    "bun ",
    timeit.timeit("bundle([hdv(10000), hdv(10000)])", number=10000, globals=globals()),
)
print(
    "bunt",
    timeit.timeit(
        "bundlet([hdvt(10000), hdvt(10000)])", number=10000, globals=globals()
    ),
)

print(
    "cos ",
    timeit.timeit(
        "cosine_similarity(hdv(10000), hdv(10000))", number=10000, globals=globals()
    ),
)
print(
    "cost",
    timeit.timeit(
        "cosine_similarityt(hdvt(10000), hdvt(10000))", number=10000, globals=globals()
    ),
)

hdv  0.5144559559994377
hdvt 0.172560273000272
bin  1.1020046439989528
bint 0.5964114729977155
bun  0.993165197000053
bunt 0.5679020170027798
cos  1.1996829909985536
cost 1.0739103449996037
